<!--
house (<u>***house_id*** </u>, name, <u>*host_id* </u>, description, room_type, latitude, longitud, post_time)

host (<u>***host_id***</u>, host_name, about, join_date, location, identity_verified, pic_url)

reviewer (<u>***reviewer_id***</u>, reviewer_name, about, join_date, location, identity_verified, pic_url)

price (<u>**house_id**</u>, **date**, price)

review (<u>**house_id**</u>, <u>**date**</u>, <u>***reviewer_id***</u>, review_name, host_name, score, comment, review_time, rating,reviewer_id, review_profile_path)   
--->





详细爬取方法参考[这篇文章](https://cloud.tencent.com/developer/article/1523321)   
[各种api详细信息](https://stevesie.com/apps/airbnb-api)

### House listing api
https://www.airbnb.com/api/v2/pdp_listing_details/42471710?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&    

 
### Review detail api
https://airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100

###  user(Host/Reviewer) api
https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en

### Price api

https://www.airbnb.com/api/v2/pdp_listing_booking_details?_format=for_web_dateless&_intents=p3_book_it&_interaction_type=pageload&_p3_impression_id=p3_1552520120_con%2FMRJw4xlFD6zi&_parent_request_uuid=a23f08cc-f28a-48d8-85e9-c05735e18ff9&currency=USD&force_boost_unc_priority_message_type=&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&show_smart_promotion=0&checkin=2021-11-04&checkout=2021-11-09&listing_id=42471710    

csv文件转为sql insert into: https://www.convertcsv.com/csv-to-sql.htm

In [1]:
import json
import os
import time
import pickle
import requests
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [2]:
# bro.get('https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-01&checkout=2021-11-04')
def getOnepage(browser, url):
    '''craw a searching page
    '''
    browser.get(url)
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source,'lxml', fromEncoding = 'ISO-8859-1')
    return browser, browser.page_source, soup
# cityPagesource = bro.page_source
def get_list(soup):
    '''return a house listing from searching page
    '''
    urlList = []
    for i in soup.find_all("div", {"class": "_qlq27g"}):
        url = 'https://airbnb.com/' + i.a['href'] + '&locale=en' # force to request english version
        urlList.append(url)
    return urlList

def extract_date(date_string):
    '''extract date'''
    return date_string.split('T')[0]

def extract_house_id(url):
    house_id = url.split('airbnb.com//rooms/')[1].split('?adults')[0]
    return house_id

In [42]:
bro = webdriver.Chrome()
urlList = []
for i in tqdm(range(0,10)):
    search_url = 'https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-11&checkout=2021-11-16&items_offset={}'.format(i*20)
    # print(search_url)
    _ , page_source, soup = getOnepage(bro, search_url)
    urls = get_list(soup)
    urlList.extend(urls)
bro.close()
print('the length of the url list is {}'.format(len(urlList)))


the length of the url list is 200


In [5]:
# urlList
# with open('./urlList.data', 'wb') as r:
    # store the data as binary data stream
#    pickle.dump(urlList, r)

In [18]:
with open('./urlList.data', 'rb') as filehandle:
    urlList = pickle.load(filehandle)
print('the length of the url list is {}'.format(len(urlList)))

house_id_list = [extract_house_id(i) for i in urlList]

the length of the url list is 155


In [33]:
## House Table
def extract_json_listing(listing_json):
    '''extract listing json
    '''
    listingDict = {'house_id':'null',
              'host_name':'null',
              'host_id':'null',
              'title':'null', 
              'address':'null',
              'latitude':'null', 
              'longtitude':'null', 
              'checkin_rating':'null',
              'clean_rating':'null',
              'comm_rating':'null',
              'satis_overall':'null',
              'bathroom_type':'null',
              'bed_type':'null',
              'bedroom_type':'null',
              'guest_type':'null',
    }
    
    listingDict['house_id'] = str(listing_json['pdp_listing_detail']['id'])
    listingDict['host_name'] = listing_json['pdp_listing_detail']['primary_host']['host_name']
    listingDict['host_id'] = int(listing_json['pdp_listing_detail']['primary_host']['id'])
    listingDict['title'] = listing_json['pdp_listing_detail']['name'] # title
    listingDict['address'] = listing_json['pdp_listing_detail']['p3_summary_address'] # address 
    listingDict['latitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lat'] # lat
    listingDict['longtitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lng'] # lng
    listingDict['checkin_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['checkin_rating'] 
    listingDict['clean_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['cleanliness_rating'] 
    listingDict['comm_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['communication_rating'] 
    listingDict['satis_overall'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['guest_satisfaction_overall']
    # JSON.pdp_listing_detail.bathroom_label
    listingDict['bathroom_type'] = listing_json['pdp_listing_detail']['bathroom_label']
    listingDict['bed_type'] = listing_json['pdp_listing_detail']['bed_label']
    listingDict['bedroom_type'] = listing_json['pdp_listing_detail']['bedroom_label']
    listingDict['guest_type'] = listing_json['pdp_listing_detail']['guest_label']
    return listingDict

listingDt = pd.DataFrame()
error_houseling_list = []
start = time.time() 
for i in tqdm(house_id_list):
    listing_url = 'https://www.airbnb.com/api/v2/pdp_listing_details/{}?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&'.format(i)
    time.sleep(1)
    response = requests.get(listing_url)
    json_input = response.json()
    try:
        dt1 = extract_json_listing(json_input)
    except:
        error_houseling_list.append(i)
    listingDt = listingDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(listingDt.shape[0], listingDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_houseling_list)))
listcol = ['house_id',
 'title', 
'address',
'latitude', 
'longtitude', 
'checkin_rating',
'clean_rating',
'comm_rating',
'satis_overall',
'bathroom_type',
'bed_type',
'bedroom_type',
'guest_type',
'host_id',
'host_name'
]
listingDt = listingDt[listcol]
listingDt.to_csv('./listingDict.csv', header=False, index = False)
listingDt


data has 155 rows and 15 columns
Appending data costs 326.89 seconds
7 data has errors


,house_id,title,address,latitude,longtitude,checkin_rating,clean_rating,comm_rating,satis_overall,bathroom_type,bed_type,bedroom_type,guest_type,host_id,host_name
0,32450232,"Metro DC Gem Close to Metro, Food & Shopping","Berwyn Heights, Maryland, United States",38.99444,-76.90879,10.0,9.0,10.0,97.0,1 bath,2 beds,1 bedroom,4 guests,149609574.0,Georgina
1,16740016,Modern Garden Apartment steps from metro,"Hyattsville, Maryland, United States",38.96453,-76.95198,10.0,10.0,10.0,99.0,1 bath,1 bed,1 bedroom,3 guests,6773843.0,Jennifer
2,32978802,Live in UM Campus Studio 01,"Hyattsville, Maryland, United States",38.97928,-76.94371,9.0,8.0,10.0,82.0,1.5 baths,1 bed,1 bedroom,2 guests,67514878.0,Lixin
3,28390867,9114 College Park Guest House Room C,"College Park, Maryland, United States",39.00678,-76.94728,10.0,10.0,10.0,99.0,1 shared bath,1 bed,1 bedroom,1 guest,190467421.0,Jacob
4,47053297,Modern Central Living 15 mins to DC w/Free Par...,"Hyattsville, Maryland, United States",38.97804,-76.96143,10.0,10.0,10.0,98.0,1 bath,2 beds,2 bedrooms,4 guests,380338110.0,Yewubdar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,21675151,Relax in Quiet DC Suburb Prime Metropolitan Local,"Silver Spring, Maryland, United States",39.02708,-76.97944,10,10,10,99,1 bath,1 bed,1 bedroom,2 guests,157777424.0,Susan
151,37452936,Modern/Spacious 3 B/rms-2.5 Baths-Silver Spring.,"Silver Spring, Maryland, United States",39.05153,-76.95293,10,10,10,99,2.5 baths,3 beds,3 bedrooms,6 guests,217630657.0,Grant
152,41123925,Great Escape near DC and UMD,"Beltsville, Maryland, United States",39.05039,-76.92214,10,10,10,93,2.5 baths,5 beds,4 bedrooms,8 guests,210989898.0,Miosotis
153,33691848,Luxury bedroom near Washington DC,"Greenbelt, Maryland, United States",39.00396,-76.91496,10,10,10,95,1 private bath,1 bed,1 bedroom,2 guests,251331868.0,Jun


,house_id,title,address,latitude,longtitude,checkin_rating,clean_rating,comm_rating,satis_overall,bathroom_type,bed_type,bedroom_type,guest_type,host_id,host_name
0,32450232,"Metro DC Gem Close to Metro, Food & Shopping","Berwyn Heights, Maryland, United States",38.99444,-76.90879,10.0,9.0,10.0,97.0,1 bath,2 beds,1 bedroom,4 guests,149609574.0,Georgina
1,16740016,Modern Garden Apartment steps from metro,"Hyattsville, Maryland, United States",38.96453,-76.95198,10.0,10.0,10.0,99.0,1 bath,1 bed,1 bedroom,3 guests,6773843.0,Jennifer
2,32978802,Live in UM Campus Studio 01,"Hyattsville, Maryland, United States",38.97928,-76.94371,9.0,8.0,10.0,82.0,1.5 baths,1 bed,1 bedroom,2 guests,67514878.0,Lixin
3,28390867,9114 College Park Guest House Room C,"College Park, Maryland, United States",39.00678,-76.94728,10.0,10.0,10.0,99.0,1 shared bath,1 bed,1 bedroom,1 guest,190467421.0,Jacob
4,47053297,Modern Central Living 15 mins to DC w/Free Par...,"Hyattsville, Maryland, United States",38.97927,-76.96141,10.0,10.0,10.0,98.0,1 bath,2 beds,2 bedrooms,4 guests,380338110.0,Yewubdar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,21675151,Relax in Quiet DC Suburb Prime Metropolitan Local,"Silver Spring, Maryland, United States",39.02528,-76.98054,10,10,10,99,1 bath,1 bed,1 bedroom,2 guests,157777424.0,Susan
151,37452936,Modern/Spacious 3 B/rms-2.5 Baths-Silver Spring.,"Silver Spring, Maryland, United States",39.05153,-76.95293,10,10,10,99,2.5 baths,3 beds,3 bedrooms,6 guests,217630657.0,Grant
152,41123925,Great Escape near DC and UMD,"Beltsville, Maryland, United States",39.05039,-76.92214,10,10,10,93,2.5 baths,5 beds,4 bedrooms,8 guests,210989898.0,Miosotis
153,33691848,Luxury bedroom near Washington DC,"Greenbelt, Maryland, United States",39.00426,-76.91537,10,10,10,95,1 private bath,1 bed,1 bedroom,2 guests,251331868.0,Jun


In [22]:
## Host Table
url = 'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
response = requests.get(url)
user_json = response.json()
user_json
# host (host_id, host_name, about, join_date, location, identity_verified, pic_url)
# reviewer (reviewer_id, reviewer_name, about, join_date, location, identity_verified, pic_url)
def extract_user(user_json, reviewer=True):
    '''extract reviewer/host and host information
    '''
    userDict = {
              'first_name':'null',
              'join_date':'null',
              'about':'null',
              'identity_verified':'null', 
              'picture_url':'null',
              'location':'null'
    }
    if reviewer:
        userDict['reviewer_id'] = user_json['user']['id']
    else:
        userDict['host_id'] = user_json['user']['id']
    userDict['first_name'] = user_json['user']['first_name']
    userDict['join_date'] = extract_date(user_json['user']['created_at'])
    userDict['about'] = user_json['user']['about']
    userDict['identity_verified'] = user_json['user']['identity_verified']
    userDict['picture_url'] = user_json['user']['picture_url']
    # userDict['created_at'] = user_json['user']['created_at']
    userDict['location'] = user_json['user']['location']
    return userDict

extract_user(user_json)

#for i in tqdm(listingDt['host_id']):
#    'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
hostDt = pd.DataFrame()
error_host_list = []
start = time.time() 
for i in tqdm(listingDt['host_id']):
    host_url = 'https://api.airbnb.com/v2/users/{}?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'.format(i)
    time.sleep(1)
    response = requests.get(host_url)
    json_input = response.json()
    try:
        dt1 = extract_user(json_input, reviewer = False)
    except:
        error_host_list.append(i)
    hostDt = hostDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(hostDt.shape[0], hostDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_host_list)))
listcol = [
    'host_id',
    'first_name',
    'location',
    'join_date',
    'about',
    'identity_verified', 
    'picture_url']
hostDt = hostDt[listcol]
hostDt.to_csv('./hostDt.csv', header=False, index = False)
hostDt


data has 155 rows and 7 columns
Appending data costs 270.96 seconds
13 data has errors


,host_id,first_name,location,join_date,about,identity_verified,picture_url
0,149609574.0,Georgina,"Berwyn Heights, MD",2017-09-07,,0.0,https://a0.muscache.com/im/pictures/user/58753...
1,6773843.0,Jennifer,"Hyattsville, MD",2013-06-06,I was born and raised in Maryland. My husband ...,1.0,https://a0.muscache.com/im/pictures/user/9cfe2...
2,67514878.0,Lixin,"University Park, MD",2016-04-17,"I am Lixin, graduated from University of Maryl...",1.0,https://a0.muscache.com/im/pictures/user/4f63d...
3,190467421.0,Jacob,"College Park, MD",2018-05-19,,0.0,https://a0.muscache.com/im/pictures/user/37a76...
4,380338110.0,Yewubdar,"Hyattsville, MD",2020-12-15,"Enjoy traveling, fun, meeting new people and l...",0.0,https://a0.muscache.com/im/pictures/user/e6ff0...
...,...,...,...,...,...,...,...
150,157777424.0,Susan,"Silver Spring, MD",2017-11-07,"Hi, I’m Susan. I enjoy the outside, and I do ...",0.0,https://a0.muscache.com/im/pictures/user/c59f6...
151,217630657.0,Grant,"Bowie, MD",2018-09-27,I am a Project Management professional and an ...,1.0,https://a0.muscache.com/im/pictures/user/8e0bb...
152,217630657.0,Grant,"Bowie, MD",2018-09-27,I am a Project Management professional and an ...,1.0,https://a0.muscache.com/im/pictures/user/8e0bb...
153,251331868.0,Jun,"Greenbelt, MD",2019-03-26,,1.0,https://a0.muscache.com/im/pictures/user/e5988...


In [23]:
## Review Table
def extract_review(json_input):
    '''iterally extract review'''
    reviewDT = pd.DataFrame()
    for rev in json_input['reviews']:
        reviewDict = {'reviewer_name':'null',
              'host_name':'null',
              'review_id':'null',
              'comments':'null', 
              'review_time':'null',
              'review_rating':'null', 
              'reviewer_id':'null'}  
        reviewDict['reviewer_name'] = rev['reviewer']['first_name'] # reviewers' first name
        reviewDict['host_name'] = rev['reviewer']['host_name'] # reviewers' first name
        reviewDict['review_id'] = str(rev['id']) # review id
        reviewDict['comments'] = rev['comments'] # comments
        reviewDict['review_time'] = extract_date(rev['created_at']) # review time
        reviewDict['review_rating'] = rev['rating'] # rating
        reviewDict['reviewer_id'] = int(rev['reviewer']['id']) # reviewer id
        # reviewDict['review_profile_path'] = rev['reviewer']['profile_path'] # profile_path
        # print(reviewDict)
        reviewDT = reviewDT.append(reviewDict, ignore_index=True)
    reviewDT['reviewer_id'] = reviewDT['reviewer_id'].astype('int64')
    return reviewDT

reviewDt = pd.DataFrame()
error_houseid_list = []
start = time.time() 
for i in tqdm(house_id_list):
    house_review_url = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id={}&role=guest&_format=for_p3&_limit=100'.format(i)
    time.sleep(1)
    response = requests.get(house_review_url)
    json_input = response.json()
    try:
        dt1 = extract_review(json_input)
        dt1['house_id'] = i
    except:
        error_houseid_list.append(i)
    reviewDt = reviewDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(reviewDt.shape[0], reviewDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_houseid_list)))
reviewCol = ['review_id','reviewer_id','house_id', 'comments', 'review_rating','host_name','review_time','reviewer_name'] 
reviewDt = reviewDt[reviewCol]
reviewDt.to_csv('./reviewDt.csv', header=False, index = False)
reviewDt


data has 5102 rows and 8 columns
Appending data costs 329.57 seconds
15 data has errors


,review_id,reviewer_id,house_id,comments,review_rating,host_name,review_time,reviewer_name
0,500717121897324240,428659519,32450232,One of the best places to stay outside of the ...,5.0,Davion,2021-11-21,Davion
1,492054249064828559,430658756,32450232,This place was very good as it relates to loca...,4.0,Jenell,2021-11-09,Jenell
2,489767755477793530,102592234,32450232,Georgina’s place was great. Wonderful location...,5.0,Michelle,2021-11-06,Michelle
3,486928912982288457,394182415,32450232,If you need a cozy room to stay then I definit...,5.0,Breana,2021-11-02,Breana
4,485444180368203436,222793167,32450232,Great space. Quiet. Perfect for a quick visit ...,5.0,Nicholas,2021-10-31,Nicholas
...,...,...,...,...,...,...,...,...
5097,449386343,11893,33691848,It was actually a great spot. Nice and Clean a...,3.0,Anthony,2019-05-06,Anthony
5098,447449508,197771963,33691848,Very clean and new place to stay. Checkin was ...,4.0,Matt,2019-05-03,Matt
5099,446928905,200258784,33691848,Great place to stay,5.0,Lyndon,2019-05-02,Lyndon
5100,439729723,196034190,33691848,"Bright, airy, and clean place. Jun is a wonder...",5.0,Kina,2019-04-19,Kina


In [25]:
## Price Tables
url = 'https://www.airbnb.com/api/v2/pdp_listing_booking_details?_format=for_web_dateless&_intents=p3_book_it&_interaction_type=pageload&_p3_impression_id=p3_1552520120_con%2FMRJw4xlFD6zi&_parent_request_uuid=a23f08cc-f28a-48d8-85e9-c05735e18ff9&currency=USD&force_boost_unc_priority_message_type=&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&show_smart_promotion=0&checkin=2021-11-04&checkout=2021-11-09&listing_id=42471710'
response = requests.get(url)
price_json = response.json()
price_json

def extract_price(price_json, houseId, thedate):
    userDict = {
              'houseId':'null',
              'Date':'null',
              'price':'null'
    }
    userDict['houseId'] = houseId
    userDict['Date'] = thedate
    userDict['price'] = price_json['pdp_listing_booking_details'][0]['bar_price']['display_prices'][0]['price_string'].split('$')[1]
    return userDict

priceDt = pd.DataFrame()
error_price_list = []
start = time.time() 
date_list = ['2021-11-23', '2021-11-24', '2021-11-25']
for house_id in tqdm(house_id_list):
        for dat in date_list:
            price_url = 'https://www.airbnb.com/api/v2/pdp_listing_booking_details?_format=for_web_dateless&_intents=p3_book_it&_interaction_type=pageload&_p3_impression_id=p3_1552520120_con%2FMRJw4xlFD6zi&_parent_request_uuid=a23f08cc-f28a-48d8-85e9-c05735e18ff9&currency=USD&force_boost_unc_priority_message_type=&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&show_smart_promotion=0&checkin={}&checkout={}&listing_id={}'.format(dat, dat, house_id)
            time.sleep(1)
            response = requests.get(price_url)
            try:
                json_input = response.json()
                dt1 = extract_price(json_input, house_id, dat)
                priceDt = priceDt.append(dt1, ignore_index = True)
            except:
                error_price_list.append(house_id)
print('data has {} rows and {} columns'.format(priceDt.shape[0], priceDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_price_list)))
priceDt.to_csv('./priceDt.csv', header=False, index = False)
priceDt


data has 447 rows and 3 columns
Appending data costs 703.79 seconds
18 data has errors


,Date,houseId,price
0,2021-11-23,32450232,48
1,2021-11-24,32450232,48
2,2021-11-25,32450232,48
3,2021-11-23,16740016,80
4,2021-11-24,16740016,80
...,...,...,...
442,2021-11-24,33691848,20
443,2021-11-25,33691848,20
444,2021-11-23,13875206,50
445,2021-11-24,13875206,50


In [31]:
## Reviewer Table
url = 'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
response = requests.get(url)
user_json = response.json()
user_json
# host (host_id, host_name, about, join_date, location, identity_verified, pic_url)
# reviewer (reviewer_id, reviewer_name, about, join_date, location, identity_verified, pic_url)
def extract_user(user_json, reviewer=True):
    '''extract reviewer/host and host information
    '''
    userDict = {
              'first_name':'null',
              'join_date':'null',
              'about':'null',
              'identity_verified':'null', 
              'picture_url':'null',
              'created_at':'null', 
              'location':'null'
    }
    if reviewer:
        userDict['reviewer_id'] = user_json['user']['id']
    else:
        userDict['host_id'] = user_json['user']['id']
    userDict['first_name'] = user_json['user']['first_name']
    userDict['join_date'] = extract_date(user_json['user']['created_at'])
    userDict['about'] = user_json['user']['about']
    userDict['identity_verified'] = user_json['user']['identity_verified']
    userDict['picture_url'] = user_json['user']['picture_url']
    # userDict['created_at'] = user_json['user']['created_at']
    userDict['location'] = user_json['user']['location']
    return userDict
#for i in tqdm(listingDt['host_id']):
#    'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
reviewerDt = pd.DataFrame()
error_reviewer_list = []
start = time.time() 
for i in tqdm(reviewDt['reviewer_id']):
    host_url = 'https://api.airbnb.com/v2/users/{}?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'.format(i)
    time.sleep(0.1)
    response = requests.get(host_url)
    json_input = response.json()
    try:
        dt1 = extract_user(json_input)
    except:
        error_reviewer_list.append(i)
    reviewerDt = reviewerDt.append(dt1, ignore_index = True)
print('data has {} rows and {} columns'.format(reviewerDt.shape[0], reviewerDt.shape[1]))
print('Appending data costs %.2f seconds'%(time.time() - start))
print('{} data has errors'.format(len(error_reviewer_list)))

listcol = ['reviewer_id','first_name','location','join_date','about','identity_verified','picture_url']
reviewerDt = reviewerDt[listcol]
reviewerDt.to_csv('./reviewerDt.csv', header=False, index = False)
reviewerDt


data has 5102 rows and 8 columns
Appending data costs 4681.61 seconds
443 data has errors


,reviewer_id,first_name,location,join_date,about,identity_verified,picture_url
0,428659519.0,Davion,"Charlotte, NC",2021-10-23,,1.0,https://a0.muscache.com/im/pictures/user/83132...
1,428659519.0,Davion,"Charlotte, NC",2021-10-23,,1.0,https://a0.muscache.com/im/pictures/user/83132...
2,102592234.0,Michelle,"Pocomoke City, MD",2016-11-05,My name is Michelle. I don’t get to do too muc...,1.0,https://a0.muscache.com/im/pictures/user/ce285...
3,394182415.0,Breana,"New York, United States",2021-03-25,,1.0,https://a0.muscache.com/im/pictures/user/b9af1...
4,222793167.0,Nicholas,"Woodbury, NJ",2018-10-27,,0.0,https://a0.muscache.com/im/pictures/user/eedb8...
...,...,...,...,...,...,...,...
5097,11893.0,Anthony,"Washington, DC",2009-03-31,I am Coming to LA To Take Photos And Need a Sp...,1.0,https://a0.muscache.com/im/users/11893/profile...
5098,197771963.0,Matt,"Ogden, UT",2018-06-24,We love traveling all over the world together.,1.0,https://a0.muscache.com/im/pictures/user/868ca...
5099,200258784.0,Lyndon,"Upper Marlboro, MD",2018-07-06,,1.0,https://a0.muscache.com/im/pictures/user/97fe4...
5100,196034190.0,Kina,"Silver Spring, MD",2018-06-16,,1.0,https://a0.muscache.com/im/pictures/user/28585...


In [32]:
reviewerDt.to_csv('./reviewerDt.csv',index = False)

In [ ]:
# Making a get request
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
# print json content
#print()
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=16465485&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
#response.json()

## response.json()['reviews'][3]['comments']
## listing-42471710 > div > div._ni9axhe > div > div._e296pg > div > div > div > a
## soup = BeautifulSoup(cityPagesource,'lxml', fromEncoding = 'ISO-8859-1')
## url.split('airbnb.com//rooms/')[1].split('?adults')[0] # 爬取room_id
## 
## soup.title.string.split('-')[0] # 爬取title
## soup.title.string.split('-')[1] # 爬取地点
## soup.title.string.split('-')[1].split(',')[-2].strip() # 州
## soup.title.string.split('-')[1].split(',')[-1].strip() # 国
## soup.title.string.split('-')[1].split(',')[-3].split('in')[-1].strip() # city
## 
## soup.find_all("a", {"class": "_105023be"})[0]['href'] # host id
## 
## list1 = []
## for i in soup.find_all("ol", {"class": "_194e2vt2"}):
##     list1.extend([i.string for i in i.find_all('span')])  # 
##     #url =  i.a['href'] # 强行改为英文网页
##     #print(url)
## list1
## soup.find_all("a", {"class": "_105023be"})[0]['href']
## Host information
## for i in soup.find_all("span", {"class": "_1ax9t0a"}):
##   print(i.get_text())
## soup.find_all("h1", {"class": "_14i3z6h"})[1].get_text() # host name
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text() # join date
## soup.find_all("span", {"class": "_1ax9t0a"})[0].get_text() # reviews number
## soup.find_all("span", {"class": "_1ax9t0a"})[1].get_text() # Identity Verification
## soup.find_all("span", {"class": "_1ax9t0a"})[6].get_text() # address 
## soup.find_all("span", {"class": "_1ax9t0a"})[4].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## response.json()['reviews'][4]['comments']